# MLFlow Hyperparameter Sweeping

**Raytheon Technologies proprietary**

Export controlled - see license file

In [ ]:
import mlflow
from providence_utils.hyperparameter_sweeper import HyperparameterSweeper
from providence.dataloaders import BasicDataloaders
from providence.training import generic_training
from providence.paper_reproductions import NasaDatasets, NasaTransformer, NasaTransformerOptimizer

# Setup MLFlow
Because there's precarious details

In [ ]:
sweeper = HyperparameterSweeper(lr=[1e-2, 3e-3, 1e-3, 3e-4], batch_size=[2**_pow for _pow in range(4, 5, 6)])

experiment_name = "Providence Sweeps Demo"
mlflow.set_experiment(experiment_name) # returns None
exp = mlflow.get_experiment_by_name(experiment_name)

## Training + MLFlow logging

In [ ]:
train_ds, test_ds = NasaDatasets()

for sweep_params in sweeper.poll_sweeps():
    # use the MLFlow run. May not be necessary to use context manager...
    with mlflow.start_run(experiment_id=exp.experiment_id):
        # log the current sweep
        mlflow.log_params(sweep_params)

        model = NasaTransformer()
        opt = NasaTransformerOptimizer(model)

        # set the learning rate through the PyTorch backdoor
        for group in opt.opt.param_groups:
            group['lr'] = sweep_params['lr']
        opt = opt._replace(batch_size=sweep_params['batch_size'])
        # changes for these parameter sweeps

        dls = BasicDataloaders(train_ds, test_ds, batch_size=opt.batch_size)
        losses = generic_training(model, opt, dls)

        # log final losses
        mlflow.log_params(
            {
                "final_training_loss": losses.training_losses[-1],
                "final_validation_loss": losses.validation_losses[-1]
            }
        )
        # log whatever else, say whether we got to the end of training
        mlflow.log_param("training_success", len(losses.training_losses) == opt.num_epochs)


Questions re: refactoring
1. Confidence towards completion of the library
2. Tutorial notebooks